In [1]:
# load the models
from cobra.io import load_json_model
ecoli = load_json_model("../../../../../ProjectNotebooks/CommunityModeling/CommPhitting/ecoli.json")
pf = load_json_model("../../../../../ProjectNotebooks/CommunityModeling/CommPhitting/pf.json")

# prevent excessive warnings
from time import process_time
import warnings
warnings.filterwarnings(action='once')
# %run ../../../modelseedpy/community/mscommscores.py

In [ ]:
from pandas import read_csv
from matplotlib import pyplot
from numpy import mean, abs, where, delete
from scipy import stats
import sigfig
import re

# mro statistics
## The selection of the maximal MRO value assumes that the interaction type is dependent upon least synergistic interaction in the pairing.
## This perspective is an order-of-magnitude less significant than the average MRO and two orders-of-magnitude less significant than the maximum MRO for each pairing because of outliers.
non_negative_mro_model1 = list(map(float, [re.search(r"(0.\d+)", x).group() for x in non_negative_interactions_df["mro_model1"]]))
    # if not isinstance(x, str):
    #     print(index, x)
        # break
non_negative_mro_model2 = list(map(float, [re.search(r"(0.\d+)", x).group() for x in non_negative_interactions_df["mro_model2"] if isinstance(x, str)]))
non_negative_mro_max = [max(individual_mros) for individual_mros in zip(non_negative_mro_model1, non_negative_mro_model2)]

negative_mro_model1 = list(map(float, [re.search(r"(0.\d+)", x).group() for x in negative_interactions_df["mro_model1"]]))
negative_mro_model2 = list(map(float, [re.search(r"(0.\d+)", x).group() for x in negative_interactions_df["mro_model2"]]))
negative_mro_max = [max(individual_mros) for individual_mros in zip(negative_mro_model1, negative_mro_model2)]

zscore = abs(stats.zscore(negative_mro_max))
negative_mro_max = delete(negative_mro_max, where(zscore > 3))
zscore = abs(stats.zscore(non_negative_mro_max))
non_negative_mro_max = delete(non_negative_mro_max, where(zscore > 3))
ttest = stats.ttest_ind(non_negative_mro_max, negative_mro_max)
print(mean(negative_mro_max), mean(non_negative_mro_max))
print(ttest.pvalue)

pyplot.rc('axes', titlesize=20, labelsize=20)
pyplot.rc('xtick', labelsize=20)
pyplot.rc('ytick', labelsize=20)
pyplot.rc('legend', fontsize=18)
fig = pyplot.figure(figsize=(7,10))

pyplot.boxplot([negative_mro_max, non_negative_mro_max], labels=["'--'", "'o/+'"])
pyplot.scatter([1]*len(negative_mro_max), negative_mro_max)
pyplot.scatter([2]*len(non_negative_mro_max), non_negative_mro_max)
pyplot.xlabel("Interaction type")
pyplot.ylabel("MRO score (overlap)")
pyplot.text(1.2,.7, f"p-value: {sigfig.round(ttest.pvalue, sigfigs=3)}", fontsize="x-large")
fig.savefig("all_v_all MRO comparion.jpg", bbox_inches='tight')


# mip statistics
non_negative_mip = list(map(float, non_negative_interactions_df["mip"]))
negative_mip = list(map(float, negative_interactions_df["mip"]))

ttest = stats.ttest_ind(non_negative_mip, negative_mip)
print(ttest.pvalue)

pyplot.rc('axes', titlesize=20, labelsize=20)
pyplot.rc('xtick', labelsize=20)
pyplot.rc('ytick', labelsize=20)
pyplot.rc('legend', fontsize=18)
fig = pyplot.figure(figsize=(7,10))

pyplot.boxplot([negative_mip, non_negative_mip], labels=["'--'", "'o/+'"])
pyplot.scatter([1]*len(negative_mip), negative_mip)
pyplot.scatter([2]*len(non_negative_mip), non_negative_mip)
pyplot.xlabel("Interaction type")
pyplot.ylabel("MIP score (syntrophy)")
pyplot.text(1.2, 6, f"p-value: {sigfig.round(ttest.pvalue, sigfigs=3)}", fontsize="x-large")
fig.savefig("all_v_all MIP comparion.jpg", bbox_inches='tight')

# biomass yield statistics
non_negative_mro_diff = non_negative_interactions_df["GRD"].to_numpy()
negative_biomass_diff = negative_interactions_df["GRD"].to_numpy()

zscore = abs(stats.zscore(negative_biomass_diff))
# print(list(map(len, [negative_biomass_diff, zscore])))
negative_biomass_diff = delete(negative_biomass_diff, where(zscore > 3))
zscore = abs(stats.zscore(non_negative_mro_diff))
non_negative_mro_diff = delete(non_negative_mro_diff, where(zscore > 3))
ttest = stats.ttest_ind(non_negative_mro_diff, negative_biomass_diff)
print(ttest.pvalue)

pyplot.rc('axes', titlesize=20, labelsize=20)
pyplot.rc('xtick', labelsize=20)
pyplot.rc('ytick', labelsize=20)
pyplot.rc('legend', fontsize=18)
fig = pyplot.figure(figsize=(7,10))

pyplot.boxplot([negative_biomass_diff, non_negative_mro_diff], labels=["'--'", "'o/+'"])
pyplot.scatter([1]*len(negative_biomass_diff), negative_biomass_diff)
pyplot.scatter([2]*len(non_negative_mro_diff), non_negative_mro_diff)
pyplot.xlabel("Interaction type")
pyplot.ylabel("Normalized difference in biomass yield (fitness)")
pyplot.text(1.2, 0.2, f"p-value: {sigfig.round(ttest.pvalue, sigfigs=3)}", fontsize="x-large")
fig.savefig("all_v_all biomass yield comparion.jpg", bbox_inches='tight')